In [1]:
import numpy as np
import pandas as pd

import pickle
import random
import os
import time
from datetime import datetime
from pathlib import Path
import glob
import json
from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from PIL import Image
plotly.offline.init_notebook_mode(connected=True)
sns.set_style('darkgrid')
#%matplotlib inline

from xyz10.io_f_mod import read_data_file
from xyz10.visualize_f_mod import visualize_trajectory, save_figure_to_image
from scipy.ndimage import median_filter
from scipy.interpolate import interp1d

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, LabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import resample, shuffle

# os.environ['CUDA_VISIBLE_DEVICES'] = "0"  # "0" = GPU_on, "-1" = GPU_off

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import TensorBoard
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#%load_ext tensorboard  # extension for notebook

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


Supporting Functions (PLOT PREDICTIONS / MAKE SUBMISSIONS)

In [2]:
def make_seq(data, mode, seq_len):  # returns sequenced data (3d-> sequence*timestep*features)
    # data: 2d -> number records*features
    # modes: "moving"/"stacked"/"full"
    if mode == "full":
         return data.reshape(1, data.shape[0], data.shape[1])
    elif mode == "stacked":
        seq_num = data.shape[0]//seq_len
        _data = data[:seq_num*seq_len]
        return _data.reshape(seq_num, seq_len, _data.shape[1])
    elif mode == "moving":
        seq_num = data.shape[0]-seq_len+1
        
        seq = []
        for i_seq in range(seq_num):
            seq.append(data[i_seq:i_seq+seq_len])
        return np.stack(seq)
    
def make_seq_inv(data, mode, seq_len):  # returns zero sequence data (2d-> timestep*1(x or y))
    # data: 3d -> number of sequences*number records-sequence length*(x or y)
    # modes: "moving"/"stacked"/"full"
    if mode == "full" or mode == "stacked":
        return data.reshape(data.shape[0]*data.shape[1], 1)
    elif mode == "moving":  # combine moving sequences via median/mean filter
        seq0_len = data.shape[0] + seq_len - 1
        _data = np.zeros((seq0_len, 1))
        
        for i_row in range(seq0_len):
            row_el = []
            j_start = max(0, i_row-seq_len+1)
            j_end = min(i_row, seq0_len-seq_len)
            
            for i_seq in range(j_start, j_end+1):
                row_el.append(data[i_seq, i_row-i_seq])
                
            _data[i_row] = np.median(row_el)

        return _data  

def make_submission(model_name, data, sufix="coarse"):

    sample_submit = pd.read_csv("./submit/sample_submission.csv")
    splits = sample_submit.site_path_timestamp.str.split(pat="_", expand=True)
    sub_data = sample_submit.copy(deep=True).join(splits)
    sub_data.rename(columns={0:"site", 1:"path", 2:"timestamp"}, inplace=True)

    for i in tqdm(list(sub_data.index)):
        site_id = sub_data.site[i]
        trace_id = sub_data.path[i]
        timestamp = sub_data.timestamp[i]

        predicted_record = data[site_id][trace_id].to_numpy()

        func_x = interp1d(predicted_record[:, 3], predicted_record[:, 0], kind="linear", copy=False, fill_value="extrapolate")
        func_y = interp1d(predicted_record[:, 3], predicted_record[:, 1], kind="linear", copy=False, fill_value="extrapolate")

        sub_data.loc[i, "x"] = func_x(timestamp)
        sub_data.loc[i, "y"] = func_y(timestamp)
        sub_data.loc[i, "floor"] = int(np.median(predicted_record[:, 2]))
        #break

    _ = [sub_data.pop(col) for col in ["site", "path", "timestamp"]]

    sub_data.to_csv(f"./submit/{model_name}_{sufix}.csv", index=False)

def plot_predictions_multi(model_name, data, sufix="coarse", delay_suffix=False):
    
    def swap_trace_floor(predicted_data):
        swap = {}

        for site_id in predicted_data.keys():

            swap[site_id] = {}
            for trace_id in predicted_data[site_id].keys():

                floor_id = predicted_data[site_id][trace_id].floor[0]
                if floor_id not in swap[site_id].keys():
                    swap[site_id][floor_id] = {}
                swap[site_id][floor_id][trace_id] = predicted_data[site_id][trace_id]

        return swap

    data = swap_trace_floor(data)
    
    floor_convert = {'5a0546857ecc773753327266': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                     '5c3c44b80379370013e0fd2b': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5d27075f03f801723c2e360f': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'},
                     '5d27096c03f801723c31e5e0': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6'},
                     '5d27097f03f801723c320d97': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5d27099f03f801723c32511d': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                     '5d2709a003f801723c3251bf': {0: '1F', 1: '2F', 2: '3F', 3: '4F'},
                     '5d2709b303f801723c327472': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F', 3: '4F'},
                     '5d2709bb03f801723c32852c': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                     '5d2709c303f801723c3299ee': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F', 3: '4F', 4: '5F', 5: '6F', 6: '7F', 7: '8F', 8: '9F'},
                     '5d2709d403f801723c32bd39': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F'},
                     '5d2709e003f801723c32d896': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5da138274db8ce0c98bbd3d2': {0: 'F1', 1: 'F2', 2: 'F3'},
                     '5da1382d4db8ce0c98bbe92e': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5da138314db8ce0c98bbf3a0': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3'},
                     '5da138364db8ce0c98bc00f1': {0: 'F1', 1: 'F2', 2: 'F3'},
                     '5da1383b4db8ce0c98bc11ab': {0: 'F1', 1: 'F2', 2: 'F3'},
                     '5da138754db8ce0c98bca82f': {0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                     '5da138764db8ce0c98bcaa46': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5da1389e4db8ce0c98bd0547': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                     '5da138b74db8ce0c98bd4774': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5da958dd46f8266d0737457b': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'},
                     '5dbc1d84c1eb61796cf7c010': {-1: 'B1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7', 7: 'F8'},
                     '5dc8cea7659e181adb076a3f': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'}}

    try:
        os.makedirs(f"./img_out/predictions/{model_name}/")
    except:
        pass

    n_s = 0
    for site_id in tqdm(data.keys()):  # over sites 
        n_s += 1
        #print(f"Processing Trajectories #{n_s}: Site-{site_id} with {len(data[site_id])} traces")

        try:
            os.makedirs(f"./img_out/predictions/{model_name}/{site_id}")
        except:
            pass

        for floor_id in data[site_id]:  # over traces
            site_path = "./data_in/metadata/" + site_id + "/"
            
            positions = []
            legends = []
            for trace_id in data[site_id][floor_id].keys():
                positions.append(data[site_id][floor_id][trace_id].to_numpy()[:, :2])
                
                if delay_suffix:
                    legends.append(f"{trace_id}_s{int(0.001*data[site_id][floor_id][trace_id].to_numpy()[0, 4])}_e{int(0.001*data[site_id][floor_id][trace_id].to_numpy()[1, 4])}")
                else:
                    legends.append(trace_id)

            try:
                floor = floor_convert[site_id][floor_id]

                meta_path = site_path + floor
                map_path = meta_path + "/floor_image.png"
                info_path = meta_path + "/floor_info.json" 

                meta_path = site_path + floor
                map_path = meta_path + "/floor_image.png"
                info_path = meta_path + "/floor_info.json" 

                with open(info_path) as info_file:
                    info_data = json.load(info_file)             

                map_width = info_data["map_info"]["width"]
                map_height = info_data["map_info"]["height"]

                fig_steps = visualize_trajectory(trajectory=positions, is_multi = True,
                                                 floor_plan_filename=map_path, mode="lines + markers", title=f"{site_id}_{floor}_{sufix}", legends=legends, 
                                                 width_meter=map_width,  height_meter=map_height)
                save_figure_to_image(fig_steps, f"./img_out/predictions/{model_name}/{site_id}/{floor}_{sufix}.png")
            except:
                print(f"Exception: wrong floor-{floor} site-{site_id}")

        #break  # only first site_id
        
def plot_predictions_steps(model_name, data, sufix="coarse", delay_suffix=False):
       
    floor_convert = {'5a0546857ecc773753327266': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                     '5c3c44b80379370013e0fd2b': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5d27075f03f801723c2e360f': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'},
                     '5d27096c03f801723c31e5e0': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6'},
                     '5d27097f03f801723c320d97': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5d27099f03f801723c32511d': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                     '5d2709a003f801723c3251bf': {0: '1F', 1: '2F', 2: '3F', 3: '4F'},
                     '5d2709b303f801723c327472': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F', 3: '4F'},
                     '5d2709bb03f801723c32852c': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                     '5d2709c303f801723c3299ee': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F', 3: '4F', 4: '5F', 5: '6F', 6: '7F', 7: '8F', 8: '9F'},
                     '5d2709d403f801723c32bd39': {-1: 'B1', 0: '1F', 1: '2F', 2: '3F'},
                     '5d2709e003f801723c32d896': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5da138274db8ce0c98bbd3d2': {0: 'F1', 1: 'F2', 2: 'F3'},
                     '5da1382d4db8ce0c98bbe92e': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5da138314db8ce0c98bbf3a0': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3'},
                     '5da138364db8ce0c98bc00f1': {0: 'F1', 1: 'F2', 2: 'F3'},
                     '5da1383b4db8ce0c98bc11ab': {0: 'F1', 1: 'F2', 2: 'F3'},
                     '5da138754db8ce0c98bca82f': {0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                     '5da138764db8ce0c98bcaa46': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5da1389e4db8ce0c98bd0547': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4'},
                     '5da138b74db8ce0c98bd4774': {-2: 'B2', -1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5'},
                     '5da958dd46f8266d0737457b': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'},
                     '5dbc1d84c1eb61796cf7c010': {-1: 'B1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7', 7: 'F8'},
                     '5dc8cea7659e181adb076a3f': {-1: 'B1', 0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7'}}

    try:
        os.makedirs(f"./img_out/predictions/{model_name}/")
    except:
        pass

    n_s = 0
    for site_id in tqdm(data.keys()):  # over sites 
        n_s += 1
        #print(f"Processing Trajectories #{n_s}: Site-{site_id} with {len(data[site_id])} traces")

        try:
            os.makedirs(f"./img_out/predictions/{model_name}/{site_id}")
        except:
            pass

        for floor_id in data[site_id]:  # over traces
            site_path = "./data_in/metadata/" + site_id + "/"
            
            positions = data[site_id][floor_id]
            legends = []

            try:
                floor = floor_convert[site_id][floor_id]

                meta_path = site_path + floor
                map_path = meta_path + "/floor_image.png"
                info_path = meta_path + "/floor_info.json" 

                meta_path = site_path + floor
                map_path = meta_path + "/floor_image.png"
                info_path = meta_path + "/floor_info.json" 

                with open(info_path) as info_file:
                    info_data = json.load(info_file)             

                map_width = info_data["map_info"]["width"]
                map_height = info_data["map_info"]["height"]

                fig_steps = visualize_trajectory(trajectory=positions, is_multi = False,
                                                 floor_plan_filename=map_path, mode="lines + markers", title=f"{site_id}_{floor}_{sufix}", legends=legends, 
                                                 width_meter=map_width,  height_meter=map_height)
                save_figure_to_image(fig_steps, f"./img_out/predictions/{model_name}/{site_id}/{floor}_{sufix}.png")
            except:
                print(f"Exception: wrong floor-{floor} site-{site_id}")
                
        #break

def xy_loss_metric(y_true, y_pred):
    e_xy = tf.sqrt(tf.square(y_true[:, 0] - y_pred[:, 0]) +  tf.square(y_true[:, 1] - y_pred[:, 1])) 
    return tf.reduce_mean(e_xy, axis=-1)

def xy_loss_metric_mse(y_true, y_pred):
    e_xy = tf.square(y_true[:, 0] - y_pred[:, 0]) +  tf.square(y_true[:, 1] - y_pred[:, 1]) 
    return tf.sqrt(tf.reduce_mean(e_xy, axis=-1))

In [3]:
def load_data(site_id, fraction_bssid, seq_len=10, data_file="10k_mix-counts"):
    print("Loading Data...")
    path = f"./data_out/full24/seq{seq_len}/"  # full24/
    file_name = f"{site_id}_{data_file}.pkl"
    
    f_list = features_list(site_id, fraction_bssid)

    return f_list, pickle.load(open(path+file_name, "rb"))[f_list]

def features_list(site_id, fraction_bssid, bssid_mode="count"):
    
    train_bssid = pickle.load(open("./data_out/train_24IDs_standardF_bssid_ranks.pkl", "rb"))
    test_bssid = pickle.load(open("./data_out/test_bssid_ranks.pkl", "rb"))

    if fraction_bssid <= 1:
        _train_bssid = train_bssid[bssid_mode][site_id].bssid.tolist()
        _test_bssid = test_bssid[bssid_mode][site_id].bssid.tolist()
        _mix = list(set(_train_bssid) & set(_test_bssid))
        _list = _mix[:int(fraction_bssid*len(_mix))]
    else:
        _train_bssid = train_bssid[bssid_mode][site_id].bssid.tolist()
        _test_bssid = test_bssid[bssid_mode][site_id].bssid.tolist()
        _mix = list(set(_train_bssid) & set(_test_bssid))
        _list = _mix[:int(fraction_bssid)]

    _list_d = []
    for record in _list:
        _list_d.append(record)
        _list_d.append(record+"_D")
        
    _list_d += ["x", "y", "f", "m", "r", "rx", "ry", "rx_cum", "ry_cum", "trace"]  # 2+7 = 9+11
        
    return _list_d

def preprocess_data(featured_data, seq_len=10, train_fraction=0.8, random_state=123):
    print(f"Processing Data of shape {featured_data[1].shape}...")

    # shuffle sequences
    data_shape = featured_data[1].shape
    columns = featured_data[1].columns.tolist()
    #display(featured_data[1].describe())

    # define train/test traces
    t_list = featured_data[1]["trace"].unique()
    t_list_s = t_list[shuffle(list(range(len(t_list))))]
    train_traces = list(t_list_s[:int((train_fraction)*len(t_list_s))])
    test_traces = list(t_list_s[int((train_fraction)*len(t_list_s)):])

    # define corresponding train/test indices
    gr = featured_data[1].groupby("trace")
    for i, trace in enumerate(train_traces):
        if i == 0:
            inds_train = gr.groups[trace]
        else:
            inds_train = inds_train.append(gr.groups[trace])

    for i, trace in enumerate(test_traces):
        if i == 0:
            inds_test = gr.groups[trace]
        else:
            inds_test = inds_test.append(gr.groups[trace])

    _ = featured_data[1].pop("trace")
    columns.remove("trace")
    
    train_data = featured_data[1].loc[inds_train, columns].copy(deep=True)
    test_data = featured_data[1].loc[inds_test, columns].copy(deep=True)

    train_data_shape = train_data.shape
    train_data_s = train_data.to_numpy().reshape(len(train_data)//seq_len, seq_len, train_data.shape[1])
    np.random.shuffle(train_data_s)
    train_data_s = pd.DataFrame(train_data_s.reshape(train_data_shape), columns=columns)
    #train_data_s[numeric_cols] = train_data_s[numeric_cols].apply(pd.to_numeric)

    test_data_shape = test_data.shape
    test_data_s = test_data.to_numpy().reshape(len(test_data)//seq_len, seq_len, test_data.shape[1])
    np.random.shuffle(test_data_s)
    test_data_s = pd.DataFrame(test_data_s.reshape(test_data_shape), columns=columns)
    #test_data_s[numeric_cols] = test_data_s[numeric_cols].apply(pd.to_numeric)
    
    shuffled_data = pd.concat([train_data_s, test_data_s], axis=0)
    
    # split-combine features and targets    
    y_x = shuffled_data.pop("x")
    y_y = shuffled_data.pop("y")
    
    x_m = shuffled_data.pop("m").to_numpy().reshape(-1,1)
    x_r = shuffled_data.pop("r").to_numpy().reshape(-1,1)
    x_rx = shuffled_data.pop("rx").to_numpy().reshape(-1,1)
    x_ry = shuffled_data.pop("ry").to_numpy().reshape(-1,1)
    x_rx_cum = shuffled_data.pop("rx_cum").to_numpy().reshape(-1,1)
    x_ry_cum = shuffled_data.pop("ry_cum").to_numpy().reshape(-1,1)
    x_f = shuffled_data.pop("f").to_numpy().astype(int)
    
    x = shuffled_data

    encoder = LabelBinarizer()#OneHotEncoder(sparse=False)
    x_f = encoder.fit_transform(x_f)
    
    #x = np.concatenate((x, x_f), axis=1)
    x = np.concatenate((x, x_m, x_r, x_rx, x_ry, x_rx_cum, x_ry_cum, x_f), axis=1)  #x_m, x_r, x_rx, x_ry, x_rx_cum, x_ry_cum, 
    y = pd.concat([y_x, y_y], axis=1).to_numpy()
    
    # split into train/validation
    train_x, val_x = x[:train_data_shape[0]], x[train_data_shape[0]:]
    train_y, val_y = y[:train_data_shape[0]], y[train_data_shape[0]:]   
            
    # scale data
    scaler = StandardScaler()  # RobustScaler()  /StandardScaler()/ MinMaxScaler
    train_x = scaler.fit_transform(train_x)
    val_x = scaler.transform(val_x)
        
    # final shaping
    train_y = train_y.reshape(train_x.shape[0]//seq_len, seq_len, 2) 
    val_y = val_y.reshape(val_x.shape[0]//seq_len, seq_len, 2)
    train_x = train_x.reshape(train_x.shape[0]//seq_len, seq_len, train_x.shape[1]) 
    val_x = val_x.reshape(val_x.shape[0]//seq_len, seq_len, val_x.shape[1])
            
    return featured_data[0][:-10], scaler, encoder, train_x, val_x, train_y, val_y  # features list is only bssid related
        

def save_models(models24, models="models24_bssid10k_SP_count_mix"):
    print("Saving Models...")
    
    idx = int(time.time())
    model_name = f"{models}_{idx}/"
    model_path = "./saved_models/"+ model_name

    if not os.path.exists(model_path):
        os.makedirs(model_path)

    for site_id in tqdm(models24.keys()):

        models24[site_id][3].save(model_path + site_id)
        with open(model_path + site_id + f"/features_list.pkl", "wb") as f:
            pickle.dump(models24[site_id][0], f)
        with open(model_path + site_id + f"/scaler.pkl", "wb") as f:
            pickle.dump(models24[site_id][1], f)
        with open(model_path + site_id + f"/f_binarizer.pkl", "wb") as f:
            pickle.dump(models24[site_id][2], f)
            
    return model_name
            
def calculate_global_metrics(comparison):

    train_mae = 0
    val_mae = 0

    for comparison_train, comparison_val in comparison:

        train_mae += comparison_train.mean_abs_error[comparison_train.index[-1]]
        val_mae += comparison_val.mean_abs_error[comparison_val.index[-1]]

    comp_length = len(comparison)
    
    train_mae /= comp_length
    val_mae /= comp_length

    print(f"Global Train/Validation MAE: {train_mae}/{val_mae}")
            
def xy_loss_metric(y_true, y_pred):
    e_xy = tf.sqrt(tf.square(y_true[:, 0] - y_pred[:, 0]) +  tf.square(y_true[:, 1] - y_pred[:, 1])) 
    return tf.reduce_mean(e_xy, axis=-1)

def xy_loss_metric_mse(y_true, y_pred):
    e_xy = tf.square(y_true[:, 0] - y_pred[:, 0]) +  tf.square(y_true[:, 1] - y_pred[:, 1]) 
    return tf.sqrt(tf.reduce_mean(e_xy, axis=-1))

In [13]:
def cont_LSTM(model, site_id, seq_len, train_x, val_x, train_y, val_y, w_f = 10, drop=0.1, learning_rate=0.005, epochs=200, batch=6):
    print(f"Continue-Fitting Model with {train_x.shape}/{val_x.shape} train/validation shapes => {round(100*train_x.shape[0]/(train_x.shape[0]+val_x.shape[0]), 1)}%...")
    print("----------------------------------------------------------------------------------------------------")
    
    epoch_iterations = 20
    
    record_shape = train_x.shape[1:]
    features_count = train_x.shape[2]
    targets_count = train_y.shape[2]
    records_num = train_x.shape[0]
        
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-7, amsgrad=False)  # "adam"
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)
    earlystop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)
    
    if batch==0:
        batch = int(np.log2(records_num/epoch_iterations))
    
    MODEL_NAME = f"S{site_id}_f{features_count}_seq{seq_len}_L2+2bi+2x{w_f}_LR{round(learning_rate*1000, 1)}_d{drop}x5_bch{batch}_{int(time.time())}"
    tensorboard = TensorBoard(log_dir=f"log/Neurals/{MODEL_NAME}", histogram_freq=1)
                
    fit = model.fit(train_x, train_y,
                validation_data=(val_x,  val_y),
                batch_size=2**batch,
                epochs=epochs,
                verbose=0,
                callbacks=[tensorboard, reduce_lr]#, earlystop]
               )
    return model

def LSTM(site_id, seq_len, train_x, val_x, train_y, val_y, w_f = 10, drop=0.1, learning_rate=0.005, epochs=200, batch=6): 
    print(f"Fitting Model with {train_x.shape}/{val_x.shape} train/validation shapes => {round(100*train_x.shape[0]/(train_x.shape[0]+val_x.shape[0]), 1)}%...")
    print("----------------------------------------------------------------------------------------------------")

    epoch_iterations = 20
    
    record_shape = train_x.shape[1:]
    features_count = train_x.shape[2]
    targets_count = train_y.shape[2]
    records_num = train_x.shape[0]
        
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-7, amsgrad=False)  # "adam"
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)
    earlystop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)
        
    if batch==0:
        batch = int(np.log2(records_num/epoch_iterations))
        
    METRICS = [
        #tf.keras.metrics.MeanAbsoluteError(name="mae", dtype=None)
        xy_loss_metric,
        #tf.keras.metrics.RootMeanSquaredError(name="rmse", dtype=None)
    ]       
       
    nodes_max = 2048
    nodes_min = 256
    nodes_num = max(min(nodes_max, int(features_count*w_f if w_f <= 1 else w_f)), nodes_min)
        
    model_DNN = keras.Sequential([       
        
        layers.Dense(nodes_num, activation="relu", input_shape=(None, features_count)),
        layers.Dropout(drop),
        layers.BatchNormalization(),
        
        layers.Dense(nodes_num, activation="relu"),  # 0.5
        layers.Dropout(drop),
        layers.BatchNormalization(),
        
        layers.Bidirectional(layers.LSTM(nodes_num, return_sequences=True), merge_mode="concat"),
        layers.Dropout(drop),
        layers.BatchNormalization(),
        
        layers.Bidirectional(layers.LSTM(nodes_num, return_sequences=True), merge_mode="concat"),
        layers.Dropout(drop),
        layers.BatchNormalization(),
                
        layers.Dense(nodes_num, activation="relu"),  # 0.5
        layers.Dropout(drop),
        layers.BatchNormalization(),
                
        layers.Dense(nodes_num, activation="relu"),  # 0.25
        
        layers.Dense(2),
    ])
    
    model_LSTM = keras.Sequential([
        
        layers.Bidirectional(layers.LSTM(int(nodes_num/10), return_sequences=True), input_shape=(None, features_count), merge_mode="concat"),
        layers.Dropout(drop),
        layers.BatchNormalization(),
        
        layers.Dense(nodes_num, activation="relu"),  # 0.5
        layers.Dropout(drop),       
        layers.BatchNormalization(),
        
        layers.Dense(nodes_num, activation="relu"),  # 0.5
        layers.Dropout(drop),
        layers.BatchNormalization(),
        
        #merge_mode="concat" "sum", "mul", "concat", "ave"
        layers.Bidirectional(layers.LSTM(int(nodes_num/10), return_sequences=True), merge_mode="concat"),
        
        layers.Dense(2),
    ])
    
    MODEL_NAME = f"S{site_id}_f{features_count}_seq{seq_len}_L2+2bi+2x{w_f}_LR{round(learning_rate*1000, 1)}_d{drop}x5_bch{batch}_{int(time.time())}"
    tensorboard = TensorBoard(log_dir=f"log/Neurals/{MODEL_NAME}", histogram_freq=1)
    
    model = model_LSTM

    model.compile(optimizer=optimizer,
                  #loss="mae",
                  loss="mse",
                  #loss=xy_loss_metric_mse,
                  metrics=METRICS)
    
    display(model.summary())  
    
    fit = model.fit(train_x, train_y,
                    validation_data=(val_x,  val_y),
                    batch_size=2**batch,
                    epochs=epochs,
                    verbose=0,
                    callbacks=[tensorboard,  reduce_lr]#, earlystop]
                   )
    return model

In [15]:
seq_len = 10

models24_xy = {}
train_val_comparison_xy = []
num_s = 0

site_ids_shapes = {"5da958dd46f8266d0737457b": (47796, 7003), "5d2709c303f801723c3299ee": (33368, 11667), "5dbc1d84c1eb61796cf7c010": (61727, 9043), "5d27075f03f801723c2e360f": (73141, 14063), 
                   "5da138b74db8ce0c98bd4774": (56668, 7075), "5dc8cea7659e181adb076a3f": (57849, 9733), "5d27096c03f801723c31e5e0": (19337, 9933), "5d2709bb03f801723c32852c": (44009, 4909), 
                   "5a0546857ecc773753327266": (26532, 6799), "5c3c44b80379370013e0fd2b": (29359, 6131), "5d27097f03f801723c320d97": (35121, 4985), "5da1382d4db8ce0c98bbe92e": (28975, 5729),
                   "5d2709b303f801723c327472": (32449, 3831), "5d2709d403f801723c32bd39": (23545, 4283), "5da138764db8ce0c98bcaa46": (27771, 3781), "5da1383b4db8ce0c98bc11ab": (34396, 3055),
                   "5d2709e003f801723c32d896": (29752, 2623), "5da138754db8ce0c98bca82f": (13795, 3259), "5da1389e4db8ce0c98bd0547": (17795, 2047), "5da138314db8ce0c98bbf3a0": (13122, 2429),
                   "5d2709a003f801723c3251bf": (9345, 2509), "5d27099f03f801723c32511d": (9700, 1855), "5da138364db8ce0c98bc00f1": (5555, 1649), "5da138274db8ce0c98bbd3d2": (6338, 985)}

site_ids = ["5dc8cea7659e181adb076a3f"]

for site_id in site_ids:#site_ids_shapes.keys():#site_ids_shapes.keys(): #site_ids:#site_ids_num_bddsid.keys(): #site_ids:
    models24_xy = {}
    
    num_s += 1
    print(f"Working on #{num_s}: Site-{site_id}")
    #========================================================================================
    
    for i_seq, seq_len in enumerate([20, 10, 5]):
        
        if i_seq == 0:  # initialize model/start fit
            f_list, scaler_xy, encoder_xy, train_x, val_x, train_y, val_y = preprocess_data(load_data(site_id, 10000, seq_len=seq_len),
                                                                                            seq_len=seq_len, train_fraction=0.97, random_state=123)
            model_xy = LSTM(site_id=site_id,
                            seq_len=seq_len,
                            train_x=train_x, val_x=val_x,
                            train_y=train_y, val_y=val_y,
                            w_f=0.3, drop=0.1, learning_rate=round(100/site_ids_shapes[site_id][0], 4), epochs=50, batch=0)
                            #w_f=0.3, drop=0.1, learning_rate=0.0056, epochs=100, batch=4)
        else: #continue fitting
            f_list, scaler_xy, encoder_xy, train_x, val_x, train_y, val_y = preprocess_data(load_data(site_id, 10000, seq_len=seq_len),
                                                                                        seq_len=seq_len, train_fraction=0.97, random_state=123)
            model_xy = cont_LSTM(model_xy,
                                 site_id=site_id,
                                 seq_len=seq_len,
                                 train_x=train_x, val_x=val_x,
                                 train_y=train_y, val_y=val_y,
                                 #w_f=0.3, drop=0.1, learning_rate=round(100/site_ids_shapes[site_id][0], 4), epochs=10+seq_len*3, batch=0)
                                 w_f=0.3, drop=0.1, learning_rate=round(100/site_ids_shapes[site_id][0], 4), epochs=50, batch=0)
    
    models24_xy[site_id] = [f_list, scaler_xy, encoder_xy, model_xy]
    #========================================================================================
    # sanity check section
    predict_columns = ["predict_X", "predict_Y"]
    real_columns = ["real_X", "real_Y"]
    col_dic = {0: "_X", 1: "_Y"}

    predictions_train_xy = pd.DataFrame(model_xy.predict(train_x).reshape(train_x.shape[0]*train_x.shape[1], 2), 
                                        columns=predict_columns)
    comparison_train_xy = pd.concat([pd.DataFrame(train_y.reshape(train_x.shape[0]*train_x.shape[1], 2), 
                                                  columns=real_columns),
                                     predictions_train_xy], axis=1)
    predictions_val_xy = pd.DataFrame(model_xy.predict(val_x).reshape(val_x.shape[0]*val_x.shape[1], 2), 
                                      columns=predict_columns)
    comparison_val_xy = pd.concat([pd.DataFrame(val_y.reshape(val_x.shape[0]*val_x.shape[1], 2), 
                                                columns=real_columns),
                                   predictions_val_xy], axis=1)

    for col_i, col in enumerate(predict_columns):
        for train_valid in [comparison_train_xy, comparison_val_xy]:
            train_valid["abs_error"+col_dic[col_i]] = np.abs(train_valid[real_columns[col_i]] - train_valid[col])

    for train_valid in [comparison_train_xy, comparison_val_xy]:
        mean_error = np.sqrt(np.power(train_valid["abs_error_X"], 2) + np.power(train_valid["abs_error_Y"], 2))
        train_valid["mean_abs_error"] = mean_error.cumsum()/train_valid.shape[0]

    print(f"Predictions (train/validation) #{num_s}: Site-{site_id}")
    #display(comparison_train_xy[["mean_abs_error"]].tail(1))
    #display(comparison_val_xy[["mean_abs_error"]].tail(1))

    train_val_comparison_xy.append([comparison_train_xy, comparison_val_xy])
    #========================================================================================
    display(comparison_val_xy.describe())
    #display(comparison_val_xy[(comparison_val_xy.abs_error_X > 20) | (comparison_val_xy.abs_error_Y > 20)])

    #comparison_val_xy[["abs_error_X", "abs_error_Y"]].plot.hist(cumulative=True, bins=100, logy=True, alpha=0.3, title=site_id)

    model_name = save_models(models24_xy, f"models24_v11_LSTM-DNN_d02_RoP_s20-10")
    keras.backend.clear_session()
    #break
print("Finished fitting")
calculate_global_metrics(train_val_comparison_xy)

#print("Finished Models Saving")
#%tensorboard --logdir log/Neurals  # commandline to start tensorboard

Working on #1: Site-5dc8cea7659e181adb076a3f
Loading Data...
Processing Data of shape (50840, 4938)...
Fitting Model with (2477, 20, 4942)/(65, 20, 4942) train/validation shapes => 97.4%...
----------------------------------------------------------------------------------------------------
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_22 (Bidirectio (None, None, 296)         6027744   
_________________________________________________________________
dropout_45 (Dropout)         (None, None, 296)         0         
_________________________________________________________________
batch_normalization_45 (Batc (None, None, 296)         1184      
_________________________________________________________________
dense_45 (Dense)             (None, None, 1482)        440154    
_________________________________________________________________
dropout_46 (Dropout)      

None

Loading Data...
Processing Data of shape (54490, 4938)...
Continue-Fitting Model with (5281, 10, 4942)/(168, 10, 4942) train/validation shapes => 96.9%...
----------------------------------------------------------------------------------------------------
Loading Data...
Processing Data of shape (56415, 4938)...
Continue-Fitting Model with (10939, 5, 4942)/(344, 5, 4942) train/validation shapes => 97.0%...
----------------------------------------------------------------------------------------------------
Predictions (train/validation) #1: Site-5dc8cea7659e181adb076a3f


,real_X,real_Y,predict_X,predict_Y,abs_error_X,abs_error_Y,mean_abs_error
count,1720.000000,1720.000000,1720.000000,1720.000000,1720.000000,1720.000000,1720.000000
mean,179.504059,111.069423,174.771759,113.354057,8.734953,9.331972,6.845131
std,44.011383,35.956020,41.098591,26.654526,10.344872,10.445747,4.059943
min,90.394128,23.687620,38.768436,45.095539,0.011586,0.007818,0.002374
25%,131.383728,100.979022,132.651325,99.808172,2.040530,2.367601,3.261399
50%,192.952146,117.587912,185.543495,117.452873,4.783585,5.006981,6.877046
75%,220.509350,137.259768,208.530800,132.795929,10.951865,11.328810,10.383263
max,249.793490,170.257281,243.848373,157.641769,73.426294,40.637811,13.736671


Saving Models...


  0%|          | 0/1 [00:00<?, ?it/s]

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./saved_models/models24_v11_LSTM-DNN_d02_RoP_s20-10_1619914768/5dc8cea7659e181adb076a3f\assets
Finished fitting
Global Train/Validation MAE: 11.874172000991674/13.736671151995067


Make Predictions

In [6]:
parsed_test_data = pickle.load(open("./data_out/full24/test-10k_mix-counts_t550.pkl", "rb"))   # counts_t550 vs counts
floor100_siteid_traceid = pickle.load(open("./data_out/floor100_siteid_traceid.pkl", "rb"))

In [17]:
model_name = "models24_v11_LSTM-DNN_d02_s20-10_1619909982"
model_path = "./saved_models/" + model_name

#models24_xy = {}
seq_len = 10
seq_mode = "moving" # ["full", "moving", "stacked"]

#site_ids = parsed_test_data.keys()
#site_ids = ["5da958dd46f8266d0737457b"]#, "5d2709c303f801723c3299ee", "5dbc1d84c1eb61796cf7c010", "5d27075f03f801723c2e360f", "5da138b74db8ce0c98bd4774"]#parsed_test_data.keys()#["5da1382d4db8ce0c98bbe92e"]

predicted_data = {}
n_s = 0
for site_id in tqdm(site_ids):#site_ids):  # over sites
    n_s += 1
    keras.backend.clear_session()
    
    print(f"Processing Predictions #{n_s}: Site-{site_id} with {len(parsed_test_data[site_id])} traces")
    ############# GET MODELS ####################
    if len(models24_xy) == 0:
        features_xy = pickle.load(open(model_path + f"/{site_id}/features_list.pkl", "rb"))
        scaler_xy = (pickle.load(open(model_path + f"/{site_id}/scaler.pkl", "rb")))
        encoder_xy = (pickle.load(open(model_path + f"/{site_id}/f_binarizer.pkl", "rb")))
        model_xy = (tf.keras.models.load_model(model_path + f"/{site_id}", custom_objects={"xy_loss_metric_mse": xy_loss_metric_mse, "xy_loss_metric": xy_loss_metric}))
    else:
        features_xy = models24_xy[site_id][0]
        scaler_xy =  models24_xy[site_id][1]
        encoder_xy = models24_xy[site_id][2]
        model_xy = models24_xy[site_id][3]
    ##############################################
    predicted_data[site_id] = {}
    for trace_id in tqdm(parsed_test_data[site_id]):  # over traces
        
        trace_record = parsed_test_data[site_id][trace_id].copy(deep=True)
        ######### GENERAL FEATURE MANIPULATION ################    
        _time = trace_record.pop("time").to_numpy()
        
        x_m = trace_record.pop("m").to_numpy().reshape(-1,1)
        x_r = trace_record.pop("r").to_numpy().reshape(-1,1)
        x_rx = trace_record.pop("rx").to_numpy().reshape(-1,1)
        x_ry = trace_record.pop("ry").to_numpy().reshape(-1,1)
        x_rx_cum = trace_record.pop("rx_cum").to_numpy().reshape(-1,1)
        x_ry_cum = trace_record.pop("ry_cum").to_numpy().reshape(-1,1)
        
        trace_record_xy = trace_record[features_xy].copy(deep=True)
        trace_record_xy = trace_record_xy.to_numpy()
        
        ##########################################################
        #  PREDICT XY (with F100 feature)
        ##########################################################
        # combine features
        _pred_f = floor100_siteid_traceid[site_id][trace_id]
        trace_record_xy_scaled = scaler_xy.transform(np.concatenate((trace_record_xy, x_m, x_r, x_rx, x_ry, x_rx_cum, x_ry_cum, encoder_xy.transform(np.full_like(_time, _pred_f))), axis=1))  #x_m, x_r, x_rx, x_ry, x_rx_cum, x_ry_cum, 
        
        folds_x, folds_y = [], []
        # make predictions on new sequenced records
        for seq_len in [20]:#[2, 5, 7, 10, 15, 20]:
        #for seq_len in range(5,21)[2, 5, 7, 10, 15, 20]:
        #for seq_len in range(5,21):
            trace_record_xy_scaled_seq = make_seq(trace_record_xy_scaled, seq_mode, seq_len)
            predictions_xy = model_xy.predict(trace_record_xy_scaled_seq)

            folds_x.append(make_seq_inv(predictions_xy[:, :, 0], seq_mode, seq_len))
            folds_y.append(make_seq_inv(predictions_xy[:, :, 1], seq_mode, seq_len))
        #print(predictions_xy.shape)

        # remove outliers (and combine different folds/seq_len)
        if len(folds_x) > 1:
            predictions_xy_x = np.median(median_filter(np.concatenate(folds_x, axis=1), (3,3)), axis=1).reshape(-1,1)
            predictions_xy_y = np.median(median_filter(np.concatenate(folds_y, axis=1), (3,3)), axis=1).reshape(-1,1)
        else:                           
            predictions_xy_x = median_filter(np.concatenate(folds_x, axis=1), (3,3))
            predictions_xy_y = median_filter(np.concatenate(folds_y, axis=1), (3,3))
                                                              
        # combine into final DataFrame
        predictions_xyf = pd.DataFrame(np.concatenate((predictions_xy_x, predictions_xy_y), axis=1), columns=["x", "y"])
        predictions_xyf["floor"] = _pred_f
        predictions_xyf["time"] = _time[:predictions_xy_x.shape[0]]

        predicted_data[site_id][trace_id] = predictions_xyf
        
        #break  # only first trace
    
    #break  # only first site_id
plot_predictions_multi(model_name, predicted_data, sufix=f"coarse_s20")

  0%|          | 0/1 [00:00<?, ?it/s]

Processing Predictions #1: Site-5da138b74db8ce0c98bd4774 with 29 traces


  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
with open(f"./submit/fit_data/{model_name}_predicted_s20.pkl", "wb") as f:
    pickle.dump(predicted_data, f)

In [8]:
predicted_data = pickle.load(open(f"./submit/fit_data/{model_name}_predicted_s20.pkl", "rb"))
plot_predictions_multi(model_name, predicted_data, sufix=f"coarse_s20")

  0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
predicted_data = pickle.load(open(f"./submit/fit_data/{model_name}_predicted_s20.pkl", "rb"))
make_submission(model_name, predicted_data, sufix=f"coarse_s20")

  0%|          | 0/10133 [00:00<?, ?it/s]